# ジョブ実行環境の設定

## 前提

このnotebookでは、Open OnDemandとSlurmの結合のための設定を行います。設定にあたっては、以下を前提とします。

* [020-フロントエンドのセットアップ](020-フロントエンドのセットアップ.ipynb)のnotebookでのセットアップが完了していること。

## 準備

### group_varsの読み込み

group_varsの読み込みに先立ち、ユニットグループ名をチェックします。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

`ugroup_name`にユニットグループ名を設定します。

In [ ]:
ugroup_name = 

group_varsを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### ansibleの動作確認

ansibleでSlurmクラスタにアクセスできることを確認します。

In [ ]:
!ansible -m ping {ugroup_name}

ansibleで指定するノード集合を設定します。

In [ ]:
target_master = f'{ugroup_name}_master'
target_compute = f'{ugroup_name}_compute'

## クラスタ設定ファイルの作成

クラスタ設定ファイルを作成し、マスターノードの`/etc/ood/config/cluster.d/`の下に配置します。

クラスタの説明を、`ood_clst_desc`に設定します。

In [ ]:
ood_cluster_desc = f'casual {ugroup_name} cluster'

Web UIからシェルでログインする先のホスト(アドレス)を設定します。以下では、シェルでのログインホストがマスターノードとなるよう設定しています。

In [ ]:
ood_shell_host = gvars['master_ipaddress']

クラスタ設定ファイルのmetadata部分のデータ構造を作成します。

In [ ]:
ood_clst_metadata = {
    'title': ood_cluster_desc
}

シェルアクセスのための設定のデータ構造を作成します。

In [ ]:
ood_clst_login = {
    'host': ood_shell_host
}

Slurmとの関連付け部分の設定を作成します。

In [ ]:
ood_clst_job = {
    'adapter': 'slurm',
    'bin':  '/usr/bin',
    'bin_overrides': {
        'sbatch': '/usr/local/bin/sbatch-wrapper'
    }
}

以上の設定をまとめ、クラスタ設定データの全体を作成します。`bin_overrides`で指定している`sbatch-wrapper`については後述します。

In [ ]:
import os
import yaml

ood_clst_file = f'{ugroup_name}_cluster.yml'

if os.path.exists(ood_clst_file):
    with open(ood_clst_file) as f:
        ood_clst_conf = yaml.safe_load(f)
else:
    ood_clst_conf = {}

if not 'v2' in ood_clst_conf.keys():
    ood_clst_conf['v2'] = {}

ood_clst_conf['v2']['metadata'] = ood_clst_metadata
ood_clst_conf['v2']['login'] = ood_clst_login
ood_clst_conf['v2']['job'] = ood_clst_job

with open(ood_clst_file, 'w') as f:
    print(f'# /etc/ood/config/clusters.d/{ood_clst_file}', file=f)
    print('---', file=f)
    yaml.safe_dump(ood_clst_conf, f)

ファイルの内容を確認します。

In [ ]:
!cat {ood_clst_file}

ファイルをマスターノードに転送します。

In [ ]:
!ansible {target_master} -b -m file -a 'path=/etc/ood/config/clusters.d state=directory'
!ansible {target_master} -b -m copy -a 'src={ood_clst_file} dest=/etc/ood/config/clusters.d/{ood_clst_file} \
    backup=true'

`sbatch-wrapper`スクリプトをマスターノードに転送します。このスクリプトでは、`${HOME}/.bashrc`を読み込んでから`/usr/bin/batch`コマンドを起動します。

このような処理をする理由について説明します。OpenHPCのジョブ実行環境は、正しく動作するための環境変数などの設定を`/etc/profile.d/`以下のスクリプトに依存しています。このため、ユーザのシェルから起動される場合には正しく動作するものの、Open OnDemandなどのような、ユーザのシェルを介さずにジョブ投入した場合には、OpenHPCに合わせた状態で`PATH`や`LD_LIBRARY_PATH`などの環境変数が設定されないため、正しく動作しません。

In [ ]:
!ansible {target_master} -b -m copy -a \
    'src=scripts/sbatch-wrapper dest=/usr/local/bin/sbatch-wrapper mode=0755'

Open OnDemandにHTTPSでアクセスするよう設定した場合には、以上でOpen OnDemandからのジョブ実行ができるようになります。

## CSRF誤検出の回避

この処理は、Open OnDemandに**HTTPSでアクセスする場合には不要であり、むしろ問題を起こす可能性があるので実施しないでください**。HTTPでアクセスする場合のみ実施します。

外部ネットワークからOpen OnDemandにHTTPでアクセスする場合、Open OnDemandが使用しているRuby on RailsがCSRF(Cross-Site Request Forgery)検出でエラーとなり動作しないため、CSRFチェックを抑止する設定を追加します。

In [ ]:
from pathlib import Path

csrfskip = 'Rails.application.config.action_controller.allow_forgery_protection = false\n'
override_app_basedir = Path('/etc/ood/config/apps')
override_apps = ['dashboard', 'myjobs']
override_destfile = 'session_store_override.rb'

for app in override_apps:
    init_d = override_app_basedir / app / 'initializers'
    !ansible {target_master} -b -m file -a \
        'path={init_d} state=directory'
    !ansible {target_master} -b -m copy -a \
        'content="{csrfskip}" dest={init_d / override_destfile}'